In [14]:
#from 
import json
from dataclasses import dataclass
import pandas as pd
import sys
import numpy as np
sys.path.append('../')
from utils import load_simple_json,load_json
from sklearn.metrics import recall_score,precision_score,confusion_matrix,accuracy_score

kfold data

In [31]:
def join_pred_data(pred_dic,data,key_word="enough"):
    re_lst=[]
    data_map = {i["id"]:idx for idx,i in enumerate(data)}
    for key in pred_dic:
        buf=None
        if int(key) in data_map.keys():
            buf = data[data_map[int(key)]]
            buf[key_word]=pred_dic[key]['predict_label']
            re_lst.append(buf)
    return re_lst

In [16]:
def join_kf_dic(in_dics,thresh=None):#majority voting
    dics=in_dics.copy()
    re_dic={}
    for dic in dics:
        for key in dic:
            if key in re_dic:
                re_dic[key]+=[dic[key]["prob"]]
            else:
                re_dic[key]=[dic[key]["prob"]]
    for key in re_dic:
        if thresh:
            re_dic[key]=sum(np.array(re_dic[key]))/len(in_dics)
        else:
            re_dic[key] = int(np.median(re_dic[key]))
    return re_dic

In [5]:
def score(data):
    label_map = {'supports':1,'refutes':1,"NOT ENOUGH INFO":0}
    gt_label = [label_map[i["label"]] for i in data]
    pred_label = [i["enough"] for i in data]
    recall,precision = recall_score(gt_label,pred_label),precision_score(gt_label,pred_label)
    print("accuracy: ",accuracy_score(gt_label,pred_label))
    print("precision: {}, recall: {}, f1: {}".format(recall,precision,(2*precision*recall)/(precision+recall)))
    print(confusion_matrix(gt_label,pred_label))

In [40]:
#TEST_DATA = load_json("../pert_large/sent/0522_base_clu_folds_PN13/test_ens5.jsonl")
TEST_DATA = load_json("../pert_large/sent/0522_base_clu_folds_PN13/all_test_clu_page_sent_ens5.jsonl")
len(TEST_DATA)

8597

In [41]:
TEST_DATA[0]

{'id': 21498,
 'claim': '雞形目的鳥腿腳強健，擅長在地面奔跑，其中有珍稀物種，體態雄健優美、顏色鮮豔；也有經濟物種，與人類關係密切。',
 'predicted_pages_1': ['雞形目'],
 'predicted_evidence': [['雞形目', 2], ['雞形目', 3], ['雞形目', 0], ['雞形目', 1]]}

In [42]:
processed_preds=[]
processed_dicts=[]
#preds = ["test_f0","test_f1","test_f2","test_f3","test_f4"]
preds = ["all_test_f0","all_test_f1","all_test_f2","all_test_f3","all_test_f4"]
#preds = ["test_f0"]
pred_dics =[]
thr_param =[]
for pred_path in preds:
    pred_dics.append(load_simple_json("../pert_large/enough/0522_nclu_neg_no_pid_folds/all_test/"+pred_path+".json"))
pred_dic = join_kf_dic(pred_dics,thresh=0.05)
pred_dic = {key:{"predict_label":pred_dic[key]} for key in pred_dic}
pair_data = join_pred_data(pred_dic,TEST_DATA)
#score(pair_data)

In [43]:
pair_data[0]

{'id': 21498,
 'claim': '雞形目的鳥腿腳強健，擅長在地面奔跑，其中有珍稀物種，體態雄健優美、顏色鮮豔；也有經濟物種，與人類關係密切。',
 'predicted_pages_1': ['雞形目'],
 'predicted_evidence': [['雞形目', 2], ['雞形目', 3], ['雞形目', 0], ['雞形目', 1]],
 'enough': 0.9632043480873108}

In [44]:
processed_preds=[]
processed_dicts=[]
#preds = ["test_f0","test_f1","test_f2","test_f3","test_f4"]
preds = ["all_test_f0","all_test_f1","all_test_f2","all_test_f3","all_test_f4"]
#preds = ["test_f0"]
pred_dics =[]
thr_param =[]
for pred_path in preds:
    pred_dics.append(load_simple_json("../pert_large/claim/0522_clu_w_noise_npid_folds/all_test/"+pred_path+".json"))
pred_dic = join_kf_dic(pred_dics,thresh=0.05)
pred_dic = {key:{"predict_label":pred_dic[key]} for key in pred_dic}
pair_data = join_pred_data(pred_dic,pair_data,key_word="predicted_label")

In [45]:
pair_data[:10]

[{'id': 21498,
  'claim': '雞形目的鳥腿腳強健，擅長在地面奔跑，其中有珍稀物種，體態雄健優美、顏色鮮豔；也有經濟物種，與人類關係密切。',
  'predicted_pages_1': ['雞形目'],
  'predicted_evidence': [['雞形目', 2], ['雞形目', 3], ['雞形目', 0], ['雞形目', 1]],
  'enough': 0.9632043480873108,
  'predicted_label': 0.009029662888497113},
 {'id': 13037,
  'claim': '教會剛建立時為解決內部的一些問題，使徒們寫下許多便條，其中有八卷不是保羅寫的為大公書信。',
  'predicted_pages_1': ['新約書信', '大公書信', '保羅_(使徒)'],
  'predicted_evidence': [['新約書信', 5],
   ['大公書信', 0],
   ['新約書信', 3],
   ['保羅_(使徒)', 3],
   ['大公書信', 10]],
  'enough': 0.5634648438543082,
  'predicted_label': 0.9773199915885925},
 {'id': 18652,
  'claim': '羅伯·昆蘭於明尼蘇達州聖保羅市出生。',
  'predicted_pages_1': ['羅伯·昆蘭'],
  'predicted_evidence': [['羅伯·昆蘭', 4], ['羅伯·昆蘭', 0], ['羅伯·昆蘭', 1]],
  'enough': 0.762176351249218,
  'predicted_label': 0.009124892391264438},
 {'id': 21378,
  'claim': '2015年美國網球公開賽女子單打比賽裡小威廉絲是上一屆的冠軍。',
  'predicted_pages_1': ['塞雷娜·威廉姆斯',
   '2015年美國網球公開賽女子單打比賽',
   '維納斯·威廉姆斯',
   '2015年溫布頓網球錦標賽',
   '2016年法國網球公開賽',
   '2003年美國網球公開賽女子單打比賽'],
 

In [281]:
def change_label(data):
    change_count=0
    enough_count=0
    for d in data:
        if d["label"]=="NOT ENOUGH INFO":
            continue
        enough_count+=1
        evidss = d["evidence"]
        preds = d["predicted_evidence"]
        at_least_get_one = False
        for evids in evidss:
            hit_count=0
            for evid in evids:
                if evid[2:] in preds:
                    hit_count+=1
            if hit_count == len(evids):
                at_least_get_one=True
        if not at_least_get_one:
            change_count+=1
            d["label"]="NOT ENOUGH INFO"
    print("changed rate: ",change_count/enough_count)
    print("at least get sent ratio: ",(enough_count-change_count)/enough_count)
                    

TEST data

In [415]:
TEST_DATA = load_json("../pert_large/sent/0522_base_clu_folds_PN13/test_ens5.jsonl")
len(TEST_DATA)

1099

In [283]:
change_label(TEST_DATA)

changed rate:  0.12790697674418605
at least get sent ratio:  0.872093023255814


In [339]:
preds = load_simple_json("../pert_large/enough/0522_fix_neg_no_pid_folds/test_f0_.json")#test_ens4_p74

In [340]:
pair_data = join_pred_data(preds,TEST_DATA)

In [341]:
score(pair_data)

accuracy:  0.7834394904458599
precision: 0.8591731266149871, recall: 0.8375314861460957, f1: 0.8482142857142858
[[196 129]
 [109 665]]


In [534]:
pair_data[0]

{'id': 21498,
 'claim': '雞形目的鳥腿腳強健，擅長在地面奔跑，其中有珍稀物種，體態雄健優美、顏色鮮豔；也有經濟物種，與人類關係密切。',
 'predicted_pages_1': ['雞形目'],
 'predicted_evidence': [['雞形目', 2], ['雞形目', 3], ['雞形目', 0], ['雞形目', 1]],
 'enough': 0.9632043480873108}

In [197]:
label_map = {'supports':1,'refutes':1,"NOT ENOUGH INFO":0}
t = [i for i in pair_data if i["label"] != "NOT ENOUGH INFO" and i["enough"]==0]
len([i for i in t if i["label"]!="supports"])

80

In [490]:
len([i for i in pair_data if i["enough"] == 1])/len(pair_data)

0.7866697685239037

In [535]:
with open("../pert_large/enough/0522_nclu_neg_no_pid_folds/all_test_clu_page_sent_nclu_e01_ens5_prob.jsonl","w",encoding="utf8",) as f:
    for i, d in enumerate(pair_data):
        f.write(json.dumps(d, ensure_ascii=False) + "\n")

In [519]:
len(pair_data)

8597